In [17]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\validation'

In [3]:
data_f=pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f=pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

In [4]:
data_f.head()

,user_id,sig_id,fakeness
0,352,1,True
1,352,2,True
2,352,3,True
3,352,4,True
4,352,5,True


In [5]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,0.000000,0.0,1.370572,0.000000,0.000000,0.0,0.149738,0.323547,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.186365,0.331796,0.356497,0.0
1,0.160537,0.000000,0.0,1.169396,0.239608,0.000000,0.0,0.000000,1.208937,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.141376,0.893188,0.000000,0.0
2,0.393000,0.223869,0.0,0.195907,0.000000,0.000000,0.0,0.060111,0.088993,0.194043,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.038160,0.215892,0.050661,0.0
3,0.280439,0.609959,0.0,0.392170,0.000000,0.568858,0.0,0.000000,0.246439,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.138333,1.180603,0.000000,0.000000,0.0
4,0.351282,0.000000,0.0,0.000000,0.000000,0.940088,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.398407,1.215286,0.000000,0.000000,0.0


In [6]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,13))]
data_f_gen_valid_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(13,25))]

In [16]:
scores_sum = 0
run_count = 0
gens_sum = 0
gens_r_count = 0
fakes_sum = 0
fakes_r_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    y_train = data_f_gen_12['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    class_weight = (data_f_gen_12.shape[0]-data_f_gen_12[y_train].shape[0])/data_f_gen_12[y_train].shape[0]
    # clf = XGBClassifier(tree_method='gpu_hist',verbosity=2, scale_pos_weight=float(class_weight))
    # clf = SVC(class_weight='balanced',gamma='scale', probability=True, kernel='rbf') # gens avg: 0.8593333333333333 fakes_avg: 0.9433333333333331 randoms_avg: 1.0 cumulative_avg: 0.9342222222222221
    clf = MLPClassifier(hidden_layer_sizes=(50,20), random_state=1923) # gens avg: 0.9416666666666668 fakes_avg: 0.8026666666666664 cumulative_avg: 0.8721666666666665
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
    y_valid_f = len(y_valid_fakes.index)*[False]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    score = clf.score(X_valid_f,y_valid_f)
    fakes_sum += score
    fakes_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']==user_id]
    y_valid_g = len(y_valid_gens.index)*[True]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    score = clf.score(X_valid_g,y_valid_g)
    gens_sum += score
    gens_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    '''
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid_rf = len(y_valid_rand_fakes.index)*[False]
    X_valid_rf = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid_rf,y_valid_rf)
    randoms_sum += score
    randoms_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    '''
    # print(scores_sum/run_count)
    print ("gens avg:", float(gens_sum)/gens_r_count, "fakes_avg:", float(fakes_sum)/fakes_r_count, 
        "cumulative_avg:", (float(gens_sum)/gens_r_count)/2+ 
        (float(fakes_sum)/fakes_r_count)/2)
print(scores_sum/run_count)
print("gens avg:", float(gens_sum)/gens_r_count, "fakes_avg:", float(fakes_sum)/fakes_r_count, 
    "cumulative_avg:", (float(gens_sum)/gens_r_count)/2+ 
    (float(fakes_sum)/fakes_r_count)/2)





  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

gens avg: 1.0 fakes_avg: 0.5666666666666667 cumulative_avg: 0.7833333333333333






  2%|#6                                                                                 | 1/50 [00:00<00:26,  1.82it/s]

gens avg: 1.0 fakes_avg: 0.6333333333333333 cumulative_avg: 0.8166666666666667






  4%|###3                                                                               | 2/50 [00:01<00:26,  1.78it/s]

...

gens avg: 0.9404761904761906 fakes_avg: 0.8006802721088433 cumulative_avg: 0.870578231292517






 98%|################################################################################3 | 49/50 [00:27<00:00,  1.75it/s]

gens avg: 0.9416666666666668 fakes_avg: 0.8026666666666664 cumulative_avg: 0.8721666666666665






100%|##################################################################################| 50/50 [00:28<00:00,  1.74it/s]

0.8721666666666669
gens avg: 0.9416666666666668 fakes_avg: 0.8026666666666664 cumulative_avg: 0.8721666666666665


In [13]:
clf.predict_proba(X_valid_f)

array([[3.77410111e-03, 9.96225899e-01],
       [2.62956356e-03, 9.97370436e-01],
       [9.31564764e-02, 9.06843524e-01],
       [9.85176601e-01, 1.48233991e-02],
       [9.92196439e-01, 7.80356128e-03],
       [9.81047384e-01, 1.89526163e-02],
       [9.99893955e-01, 1.06045376e-04],
       [9.91656827e-01, 8.34317263e-03],
       [9.94328969e-01, 5.67103109e-03],
       [9.99998098e-01, 1.90236809e-06],
       [9.99988006e-01, 1.19939700e-05],
       [9.99984664e-01, 1.53358585e-05],
       [9.99999960e-01, 3.99752390e-08],
       [9.99999775e-01, 2.24635721e-07],
       [9.99999787e-01, 2.12685029e-07],
       [9.99997704e-01, 2.29568339e-06],
       [9.99998405e-01, 1.59492152e-06],
       [9.99992893e-01, 7.10733169e-06],
       [9.99992893e-01, 7.10733169e-06],
       [3.74436983e-01, 6.25563017e-01],
       [9.51167760e-01, 4.88322395e-02],
       [9.99992893e-01, 7.10733169e-06],
       [9.99097763e-01, 9.02237166e-04],
       [9.98972042e-01, 1.02795832e-03],
       [9.999698

In [ ]:
X_valid_f.shape